In [ ]:
###SASSIE HYDROGRAPHY BUOY DATA USE AND VISUALIZATION##

#SOURCE: https://github.com/NASA-SASSIE/data-vis/
#REFERENCE: (input doi for the running narrative here)
#CREATOR NAME: Elizabeth Westbrook
#CREATOR EMAIL: westbrooke@uncw.edu


import numpy as np
import xarray as xr
xr.set_options(display_max_rows = 100,display_width = 200)
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
import requests

##########ENTER YOUR EARTHDATA USERNAME AND PASSWORD#############
username = 'Enter your Earth Data username here'
password = 'Enter your Earth Data password here'
earthdata = requests.auth.HTTPBasicAuth(username, password)

############ CHANGE LOCAL DIRECTORIES FOR YOUR DEVICE ############## 
 
##LOCAL DIRECTORY TO HYDROBUOY DATA
dir_in = 'Enter local directory where you would like SASSIE data to be saved' #change
##LOCAL DIRECTORY FOR AMSR SEA ICE DATA TO BE STORED
ice_dir =  'Enter local directory where you would like ice data to be saved' #change 
##LOCAL DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Enter local directory where you would like SASSIE shiptrack data to be saved'  #change
##LOCAL DIRECTORY TO SAVE FIGURES
fig_dir = 'Enter local directory where you would like figures to be saved' #change

In [ ]:
############ DO NOT CHANGE THIS BLOCK ####################

##SASSIE DATA DIR
if not os.path.exists(dir_in):
    os.makedirs(dir_in)

##DOWNLOAD DATA FROM PO.DAAC
    buoy_numbers = ['01','02','03','04','05','06','07','08','09','10','11']
    for number in buoy_numbers:
        filename = 'SASSIE_Fall_2022_UpTempO_'+number+'_V1.nc'
        if os.path.isfile(dir_in+filename)==False:
            url = requests.get('https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_DRIFTER_UPTEMPO_V1/'+filename, auth=earthdata)
            with open(dir_in+filename, 'wb') as f:
                f.write(url.content)
                print('Saving Input File: '+dir_in+filename)
        else:
            print('File already in local directory')

#ICE DIR
if not os.path.exists(ice_dir):
    os.makedirs(ice_dir)
    
##SHIP DATA DIR
if not os.path.exists(ship_dir):
    os.makedirs(ship_dir)
    url = requests.get('https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc',auth=earthdata)
    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
        f.write(url.content) 
    print('Saving Input File: ' +ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc')
else: 
    print('Ship Track file already in local directory')
                  
    
#FIGURE DIR
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

##READ IN BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url)

################# DO NOT CHANGE THIS BLOCK ###############

In [ ]:
##LOAD IN THE NETCDF FILES AND LOOK AT THEIR METADATA 
##UNCOMMENT PRINT STATEMENTS TO SEE INFORMATION##

files = sorted(glob.glob(dir_in + '/*.nc'))
for file in files:
    #print(file)
    
    ##SHOWS THE GLOBAL METEADATA AND WHICH VARIABLES ARE IN THE FILE
    ds = xr.open_dataset(file)
    #print(ds)
    
    ##SHOW THE VARIABLE ATTRIBUTES FOR ANY GIVEN VARIABLE WITHIN THE FILE. Replace "salinity" with your variable of interest:
    variable_info = ds.time
    #print(variable_info.values)

In [ ]:
##USE THIS TO PLOT THE UPTEMPO TRACKS ON A MAP. ENTER 'all_data' TO MAP ALL UPTEMPO DATA AND 'campaign_data' TO MAP ONLY DATA TAKEN DURRING CAMPIGN.  
def map_uptempo(map_view,ship_track=True,bathymetry_data=True):
    
    ##TIME DATA FROM BOUY 01 (deployed on September 9th and was still collecting data as of 03/13) 
    files = sorted(glob.glob(dir_in + '/*.nc'))
    ds = xr.open_dataset(files[3])
    time = np.squeeze(ds['time'].values)
    start_time = min(time)
    end_time = max(time)
        
    ##CREATE A MAP WITH LAND AND CITY MARKERS AND SHIP TRACK  
    if map_view == 'all_data':
        latmin = 70
        latmax =80
        lonmin=-177
        lonmax=-145
        var_min = start_time.astype('int64')
        var_max =end_time.astype('int64')
    if map_view == 'campaign_data':
        latmin = 70
        latmax =74
        lonmin=-157
        lonmax=-145
        ds_ship = xr.open_dataset(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        var_min = min(ship_time.values).astype('int64')
        var_max = max(ship_time.values).astype('int64')
        
    campaign_start = datetime(2022,9,8,8)
    campaign_end = datetime(2022,9,29)
        
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50')
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')

    
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3)
    if map_view =='all_data':
        ax.set_title('HydroBuoy Tracks',fontsize=20,pad=1) 
    if map_view == 'campaign_data':
        ax.set_title('HydroBuoy Tracks Throughout SASSIE Campaign',fontsize=15,pad=1.5) 
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,82,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,180,2))
    gl.top_labels = False
    gl.bottom_labels = True
    gl.right_labels = False
    
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
    plt.legend(loc=3)
    
    #OPTIONAL MAP ADD ONS: 
    if ship_track==True:
        #ADD SHIP TRACK DATA
        ds_ship = xr.open_dataset(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
        track = ax.plot(ship_lon, 
                     ship_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)
        plt.legend(loc=3)
    
    if bathymetry_data == True:    
        ##ADD BATHYMETRY DATA
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)

        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)
        
    
    ##APPLY THE TIME AND LOCATION DATA IN EACH UPTEMPO FILE 
    files = sorted(glob.glob(dir_in + '/*.nc'))
    if map_view == 'all_data':
        for file in files:
            ds_ut = xr.open_dataset(file)
            ut_time = np.squeeze(ds_ut['time'])
            ut_lat = np.squeeze(ds_ut['latitude'])
            ut_lon = np.squeeze(ds_ut['longitude'])

            deployment_track = ax.scatter(ut_lon,ut_lat,s = 1,
                           c = ut_time,cmap = 'jet',
                           transform=cartopy.crs.PlateCarree(),zorder=2,vmin=var_min,vmax=var_max)



        cbar = fig.colorbar(deployment_track, ax=ax, orientation="horizontal", pad=0.1)
        cbar.set_label(label='Date',size='large',weight='bold')
        cbar_tick_array=(np.linspace(start_time.astype('int64'),end_time.astype('int64'),5))
        cbar.set_ticks(cbar_tick_array)
        cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)  
    if map_view == 'campaign_data':
        for file in files:
            ds_ut = xr.open_dataset(file)
            ds_ut_campaign = ds_ut.sel(time=slice(campaign_start,campaign_end))
            ut_time = np.squeeze(ds_ut_campaign['time'])
            ut_lat = np.squeeze(ds_ut_campaign['latitude'])
            ut_lon = np.squeeze(ds_ut_campaign['longitude'])

            deployment_track = ax.scatter(ut_lon,ut_lat,s = 1,
                           c = ut_time,cmap = 'jet',
                           transform=cartopy.crs.PlateCarree(),zorder=2,vmin=var_min,vmax=var_max)



        cbar = fig.colorbar(deployment_track, ax=ax, orientation="horizontal", pad=0.1)
        cbar.set_label(label='Date',size='large',weight='bold')
        cbar_tick_array=(np.linspace(min(ship_time.values).astype('int64'),max(ship_time.values).astype('int64'),5))
        cbar.set_ticks(cbar_tick_array)
        cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)  
  
    #SAVE FIGURE
    if not os.path.exists(fig_dir+'maps/'):
        os.makedirs(fig_dir+'maps/')
    print('Saving Output Image:  '+fig_dir+'maps/UpTempO_Buoys'+'_'+map_view+'.png')
    plt.savefig(fig_dir+'maps/UpTempO_Buoys'+'_'+map_view+'.png',dpi='figure',format='png')
    

In [ ]:
map_uptempo('all_data',ship_track=False)   
map_uptempo('campaign_data')   

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET
def define_var_attributes(var):
    if var =='salinity':
        colormap = 'viridis'
        var_label = 'Salinity'
    if var =='temperature':
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    return colormap,var_label

In [ ]:
## USE FOR DATA VARIABLES WITH A DEPTH DIMENSION 
def plot_uptempo_data(buoy,var,var_min,var_max):
    
    colormap,var_label = define_var_attributes(var)

    #GET UPTEMPO DATA
    file = files[buoy-1]
    ds = xr.open_dataset(file)
    dates = pd.to_datetime(ds.time.values).date
    deployment_date = dates[0]
    lat = np.squeeze(ds['latitude'])
    lon = np.squeeze(ds['longitude'])
    
    #Since the temp and salinity variables are different in each file, this searches for temp and salinity at each possible depth
    #and creates a dictionary entry if that variable exists. 
    var_dict= {}
    depth_dict={}
    if var == 'temperature':
        possible_depths = ['0014','0044','0500','6000','5000','4000','3500','3000','2500','2000','1500','1000','0750','0250','0025','0075']
        for depth in possible_depths:
            try:
                var_dict['ut_temp'+depth] = np.squeeze(ds['temperature'+depth].values)
                depth_dict['ut_depth'+depth] = float(depth[0:2]+'.'+depth[2:4])
            except:
                pass
    if var == 'salinity':
        possible_depths = ['0038','6000','4000','3000','2000','1000','0500','0051']
        for depth in possible_depths:
            try:
                var_dict['ut_sal'+depth] = np.squeeze(ds['salinity'+depth].values)
                depth_dict['ut_depth'+depth] = float(depth[0:2]+'.'+depth[2:4])
            except:
                pass  
    sorted_depth_dict = dict(sorted(depth_dict.items(), key=lambda x:x[0]))
    sorted_var_dict = dict(sorted(var_dict.items(), key=lambda x:x[0]))
   
    ##CONFIGURE UPTEMPO DATA INTO NUMPY ARRAYS
    number_of_measurements = len(depth_dict)
    depths = list(sorted_depth_dict.values())
    var_at_depths = list(sorted_var_dict.values())
    
    var_arr = np.zeros((lat.size,number_of_measurements))
    z = np.zeros((lat.size,number_of_measurements))
    
    #Bouys 2,6, and 7 took salinity measurements at only one depth, 
    #so some of the steps used for other cases are unecessary and cause exception
    if (var == 'salinity')&((buoy==2)|(buoy==6)|(buoy==7)):
        all_lat = lat.values
        all_lon = lon.values
        flat_z = np.squeeze(np.full((lat.size,1), depths))
        flat_arr = var_at_depths
    else:
        for i in np.arange(0,number_of_measurements,1):
            z[:,i] = np.squeeze(np.full((lat.size,1), depths[i]))
            var_arr[:,i] = var_at_depths[i]
        flat_z = z.flatten(order='F')
        flat_arr = var_arr.flatten(order='F')
        all_lon = np.squeeze(np.tile(lon.values,(1,number_of_measurements)))
        all_lat = np.squeeze(np.tile(lat.values,(1,number_of_measurements)))
        
    ##CREATE THE 3D PLOT 
    latmin = min(lat.values)
    latmax = max(lat.values)
    lonmin = min(lon.values)
    if (buoy == 4)|(buoy==9): 
        lonmax = -150
    else: 
        lonmax = max(lon.values)

    if (var == 'salinity')&((buoy==2)|(buoy==6)|(buoy==7)):
        dmin = 0
        dmax = 0.5
    else:
        dmin = min(depths)
        dmax = max(depths)
        
   
    fig = plt.figure(figsize=(12,10))
    ax = plt.axes(projection='3d',computed_zorder=False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    ax.set_xlim(lonmin, lonmax); ax.set_ylim(latmin,latmax); ax.set_zlim(dmin,dmax);
    ax.get_xaxis().get_major_formatter().set_useOffset(False)
    xticks = list(np.linspace(lonmin,lonmax,4))
    ax.set_xticks(np.around(xticks,2))
    ax.set_xticklabels(np.around(xticks,2),rotation=30, ha='center',va='center', minor=False,size='small')
    yticks = list(np.linspace(latmin,latmax,4))
    ax.set_yticks(np.around(yticks,2))
    ax.invert_yaxis()
    ax.set_yticklabels(np.around(yticks,2),rotation=-15,va='bottom', minor=False,size='small')

    ax.set_zticks(depths)
    ax.set_zticklabels(np.around(depths,2),ha='center',va='center', minor=False,size='small')

    ax.set_xlabel('Longitude',fontsize=13,labelpad=10, fontweight='bold')
    ax.set_ylabel('Latitude',fontsize=13,labelpad = 7,fontweight='bold')
    ax.set_zlabel('Depth (m)',fontsize=13, labelpad = 16,fontweight='bold')
    ax.view_init(195, 280)

    ttl = ax.set_title('Buoy'+ ' '+str(buoy)+' '+var_label+' Measurements',fontsize=20,y=0.92,x=0.6)
    
    fig.canvas.draw()
    plt.tight_layout()


    #APPLY UPTEMPO DATA
    a = ax.scatter(all_lon,all_lat,flat_z,c=(flat_arr),cmap = colormap,s=0.5,zorder=1,vmin = var_min,vmax = var_max)
    
    cbar = fig.colorbar(a,shrink=0.6)
    cbar.set_label(label=var_label,size=15,weight='bold',labelpad=10)
   
        
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+var):
        os.makedirs(fig_dir+var)
    print('Saving Output Image:  '+fig_dir+var+'/UpTempO_Buoy'+'_'+str(buoy)+'_'+var+'.png')
    plt.savefig(fig_dir+var+'/UpTempO_Buoy'+'_'+str(buoy)+'_'+var+'.png',dpi='figure',format='png',bbox_inches='tight')
    

In [ ]:
plot_uptempo_data(3,'temperature',-1.5,1.5)
plot_uptempo_data(3,'salinity',20,30)